In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
dataset = pd.read_csv("housing.csv")

inputs = dataset[["housing_median_age","total_rooms","total_bedrooms","households","median_income"]]
print(inputs)
targets = dataset[["population","median_house_value"]]
print(targets)


    housing_median_age  total_rooms  total_bedrooms  households  median_income
0                 41.0        880.0           129.0       126.0         8.3252
1                 21.0       7099.0          1106.0      1138.0         8.3014
2                 52.0       1467.0           190.0       177.0         7.2574
3                 52.0       1274.0           235.0       219.0         5.6431
4                 52.0       1627.0           280.0       259.0         3.8462
5                 52.0        919.0           213.0       193.0         4.0368
6                 52.0       2535.0           489.0       514.0         3.6591
7                 52.0       3104.0           687.0       647.0         3.1200
8                 42.0       2555.0           665.0       595.0         2.0804
9                 52.0       3549.0           707.0       714.0         3.6912
10                52.0       2202.0           434.0       402.0         3.2031
11                52.0       3503.0           752.0 

In [3]:
inputs = np.array(inputs, dtype='float32')
for i in inputs:
    i[4]= i[4]*100

targets = np.array(targets, dtype='float32')
for i in targets:
    i[1]= i[1]/100

In [12]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(targets)

TypeError: expected np.ndarray (got Tensor)

In [5]:
#training_dataset
train_ds = torch.utils.data.TensorDataset(inputs, targets)

batch_size = 5
train_dl = torch.utils.data.DataLoader(train_ds, batch_size, shuffle=True)

i = 1
for input_batch, target_batch in train_dl:
    print("Batch :", i)
    print(input_batch)
    print(target_batch)
    i += 1


Batch : 1
tensor([[  52.0000, 2491.0000,  474.0000,  468.0000,  307.5000],
        [  52.0000, 3549.0000,  707.0000,  714.0000,  369.1200],
        [  49.0000, 1916.0000,  447.0000,  378.0000,  192.7400],
        [  42.0000, 1639.0000,  367.0000,  366.0000,  171.3500],
        [  52.0000, 1274.0000,  235.0000,  219.0000,  564.3100]])
tensor([[1098., 2135.],
        [1551., 2611.],
        [ 863., 1223.],
        [ 929., 1598.],
        [ 558., 3413.]])
Batch : 2
tensor([[  52.0000, 2535.0000,  489.0000,  514.0000,  365.9100],
        [  52.0000,  729.0000,  160.0000,  155.0000,  168.7500],
        [  21.0000, 7099.0000, 1106.0000, 1138.0000,  830.1400],
        [  52.0000, 1467.0000,  190.0000,  177.0000,  725.7400],
        [  52.0000, 3503.0000,  752.0000,  734.0000,  327.0500]])
tensor([[1094., 2992.],
        [ 395., 1320.],
        [2401., 3585.],
        [ 496., 3521.],
        [1504., 2418.]])
Batch : 3
tensor([[  52.0000, 2436.0000,  541.0000,  478.0000,  172.5000],
        [  

In [6]:
model = torch.nn.Linear(5,2)
print(model.weight)
print(model.bias)
list(model.parameters())

Parameter containing:
tensor([[-0.0489,  0.0765,  0.0442, -0.1435, -0.0545],
        [ 0.2803,  0.1374,  0.0512, -0.3154, -0.3342]], requires_grad=True)
Parameter containing:
tensor([ 0.1121, -0.0612], requires_grad=True)


[Parameter containing:
 tensor([[-0.0489,  0.0765,  0.0442, -0.1435, -0.0545],
         [ 0.2803,  0.1374,  0.0512, -0.3154, -0.3342]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1121, -0.0612], requires_grad=True)]

In [7]:
loss_fn = torch.nn.functional.mse_loss

In [8]:
#optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-7)

In [9]:
def train(num_epochs, model, loss_function, training_dataloader):
    for epoch in range(num_epochs):
        for input_batch, target_batch in training_dataloader:
            
            pred = model(input_batch)
            loss = loss_function(pred, target_batch)

            loss.backward()
            
            opt.step()
            opt.zero_grad()
            
        if loss.item()< 5000:
            print(loss.item())
            print(pred)
            print(model.weight)
            print(model.bias)

In [10]:
train(20000, model, loss_fn, train_dl)

4729.23486328125
tensor([[1514.0844, 2462.0469],
        [1469.9332, 2569.5269],
        [ 653.0056, 1628.3613],
        [ 428.8909, 1883.5319],
        [ 879.8331, 1155.2767]], grad_fn=<AddmmBackward>)
Parameter containing:
tensor([[ 2.7815,  0.1505,  0.7651,  0.4759, -0.1632],
        [ 4.9484, -0.3878,  2.1807,  0.1870,  5.3960]], requires_grad=True)
Parameter containing:
tensor([ 0.1695, -0.0011], requires_grad=True)
4783.6064453125
tensor([[ 978.3007, 1480.4398],
        [ 692.4753, 1543.4081],
        [ 618.4153, 1445.3613],
        [ 917.2300, 1280.0093],
        [ 825.2886, 1504.5469]], grad_fn=<AddmmBackward>)
Parameter containing:
tensor([[ 2.9124,  0.1571,  0.7012,  0.5084, -0.1714],
        [ 5.1131, -0.5333,  2.0936,  0.1844,  5.3528]], requires_grad=True)
Parameter containing:
tensor([ 0.1730, -0.0016], requires_grad=True)
4297.19287109375
tensor([[ 959.2810, 1534.2592],
        [ 420.4493, 1275.0067],
        [ 777.0987, 1211.4509],
        [1021.1812, 2083.3506],
      